In [14]:
import sympy as sp
from IPython.display import display, clear_output

#--------------- MAIN DEFINITIONS ---------------
#Time
t = sp.Symbol('t', real=True);
# ........................
# Coordinates variables
x0 = sp.Symbol('x0', real=True);
y0 = sp.Symbol('y0', real=True);
z0 = sp.Symbol('z0', real=True);

x1 = sp.Symbol('x1', real=True);
y1 = sp.Symbol('y1', real=True);
z1 = sp.Symbol('z1', real=True);

x2 = sp.Symbol('x2', real=True);
y2 = sp.Symbol('y2', real=True);
z2 = sp.Symbol('z2', real=True);

l = sp.Symbol('l', real=True); #magnitude of the vector distance from frame 1 to frame 4 (modulus)
d = sp.Symbol('d', real=True); #magnitude of the vector distance from frame 2 to frame 3 (modulus)

alpha_L = sp.Function('alpha_L', real=True); #rotation angle of the left wheel
alpha_R = sp.Function('alpha_R', real=True); #rotation angle of the right wheel

#Masses
m_b = sp.Symbol('m_b', real=True); #body mass
m_w = sp.Symbol('m_w', real=True); #wheel mass

g = sp.Symbol('g', real=True); #gravity
r = sp.Symbol('r', real=True); #wheel radius
# ........................
# Robot configuration variables (CONFERIR)
# p_c = sp.Function('p_c', real=True); #CONFERIR
x_c = sp.Function('x_c', real=True); #CONFERIR
y_c = sp.Function('y_c', real=True); #CONFERIR

phi= sp.Function('phi', real=True);
theta = sp.Function('theta', real=True);

 #CONFERIR
# q = sp.Matrix([[p_c(t)],\
#               [theta(t)],\
#               [phi(t)]]
#               )
# q = sp.Matrix([[x_c(t)],\
#               [y_c(t)],\
#               [theta(t)],\
#               [phi(t)],\
#               [alpha_L(t)],\
#               [alpha_R(t)]],\

#               )

q = sp.Matrix([[x_c(t)],\
              [y_c(t)],\
              [phi(t)],\
              [theta(t)],\
              [alpha_R(t)],\
             [alpha_L(t)]],\

              )

q_dot = sp.simplify(sp.diff(q,t))
# ........................
#Inertia matrices
Iw1 = sp.Symbol('K', real=True); #MUDAR!
Iw2 = sp.Symbol('J', real=True);
Iw3 = sp.Symbol('K', real=True);

Ib1 = sp.Symbol('Ib1', real=True);
Ib2 = sp.Symbol('Ib2', real=True);
Ib3 = sp.Symbol('Ib3', real=True);

I_w = sp.Matrix([[Iw1, 0, 0],\
                [0, Iw2, 0],\
                [0, 0, Iw1]])

I_b = sp.Matrix([[Ib1, 0, 0],\
                [0, Ib2, 0],\
                [0, 0, Ib3]])
# ........................



In [15]:
def get_rotation(var, axis:str):
  if not axis in ['x', 'y', 'z']:
    return None
  if axis == 'x':
    R = sp.Matrix([[1, 0, 0],\
                [0, sp.cos(var), -sp.sin(var)],\
                [0, sp.sin(var), sp.cos(var)]
                   ])
  if axis == 'y':
    R = sp.Matrix([[sp.cos(var),0, sp.sin(var)],\
                [0, 1, 0],\
                [-sp.sin(var), 0, sp.cos(var)]])
  if axis == 'z':
    R = sp.Matrix([[sp.cos(var), -sp.sin(var), 0],\
                [sp.sin(var), sp.cos(var), 0],\
                [0, 0, 1]])
  R_dot = sp.simplify(sp.diff(R, t))
  Sw = sp.simplify(R_dot*sp.transpose(R))
  w = sp.Matrix([[Sw[2,1]],\
                [-Sw[2,0]],\
                 [Sw[1,0] ]])
  return [R, R_dot, Sw, w]

#--------------- ROTATION MATRICES ---------------
#R1_0 = R(z,phi)
R1_0, R1_0_dot, Sw1_0, w1_0 = get_rotation(phi(t), 'z')
display('R1_0: ', R1_0)
display('R1_0_dot: ', R1_0_dot)
display('Sw1_0: ', Sw1_0)
display('w1_0: ', w1_0)
print('-----------------------------------------')
# ........................

#R4_1 = R(y,theta)
R4_1, R4_1_dot, Sw4_1, w4_1 = get_rotation(theta(t), 'y')
display('R4_1: ', R4_1)

#R4_0 = R1_0.R4_1
R4_0 = R1_0*R4_1
R4_0_dot = sp.simplify(sp.diff(R4_0, t))
Sw4_0 = sp.simplify(sp.transpose(R4_0)*R4_0_dot) #sp.simplify(R4_0_dot*sp.transpose(R4_0))
w4_0 = sp.Matrix([[Sw4_0[2,1]],\
              [-Sw4_0[2,0]],\
                [Sw4_0[1,0] ]])
display('R4_0: ', R4_0)
display('R4_0_dot: ', R4_0_dot)
display('Sw4_0: ', Sw4_0)
display('w4_0: ', w4_0)
print('-----------------------------------------')
# ........................

#R2_1 = R(y,alpha_L)
R2_1 = get_rotation(alpha_R(t), 'y')[0] #CONFERIR
#R2_1 = sp.Identity(3)

#R2_0 = R2_0.R2_1
R2_0 = R1_0*R2_1
R2_0_dot = sp.simplify(sp.diff(R2_0, t))
#R2_0_dot = R2_0_dot.subs(sp.diff(alpha_R(t), t), (P2_0_dot[0]*sp.cos(phi(t)) + P2_0_dot[1]*sp.sin(phi(t)))/r) #REMOVER
Sw2_0 = sp.simplify(sp.transpose(R2_0)*R2_0_dot) #CONFERIR
w2_0 = sp.Matrix([[Sw2_0[2,1]],\
              [-Sw2_0[2,0]],\
                [Sw2_0[1,0] ]])

display('R2_0: ', R2_0)
display('R2_0_dot: ', R2_0_dot)
display('Sw2_0: ', Sw2_0)
display('w2_0: ', w2_0)
print('-----------------------------------------')
# ........................

#R3_1 = R(y,alpha_L)
R3_1 = get_rotation(alpha_L(t), 'y')[0] #CONFERIR
#R3_1 = sp.Identity(3)

#R3_0 = R3_0.R3_1
R3_0 = R1_0*R3_1
R3_0_dot = sp.simplify(sp.diff(R3_0, t))
Sw3_0 = sp.simplify(sp.transpose(R3_0)*R3_0_dot) #
w3_0 = sp.Matrix([[Sw3_0[2,1]],\
              [-Sw3_0[2,0]],\
                [Sw3_0[1,0] ]])
display('R3_0: ', R3_0)
display('R3_0_dot: ', R3_0_dot)
display('Sw3_0: ', Sw3_0)
display('w3_0: ', w3_0)
print('-----------------------------------------')
# ........................
#------------------------------------------------

'R1_0: '

Matrix([
[cos(phi(t)), -sin(phi(t)), 0],
[sin(phi(t)),  cos(phi(t)), 0],
[          0,            0, 1]])

'R1_0_dot: '

Matrix([
[-sin(phi(t))*Derivative(phi(t), t), -cos(phi(t))*Derivative(phi(t), t), 0],
[ cos(phi(t))*Derivative(phi(t), t), -sin(phi(t))*Derivative(phi(t), t), 0],
[                                 0,                                  0, 0]])

'Sw1_0: '

...

'Sw3_0: '

Matrix([
[                                    0, -cos(alpha_L(t))*Derivative(phi(t), t),             Derivative(alpha_L(t), t)],
[cos(alpha_L(t))*Derivative(phi(t), t),                                      0, sin(alpha_L(t))*Derivative(phi(t), t)],
[           -Derivative(alpha_L(t), t), -sin(alpha_L(t))*Derivative(phi(t), t),                                     0]])

'w3_0: '

Matrix([
[-sin(alpha_L(t))*Derivative(phi(t), t)],
[             Derivative(alpha_L(t), t)],
[ cos(alpha_L(t))*Derivative(phi(t), t)]])

-----------------------------------------


In [16]:
#--------------- Kinematic Maps ---------------
#Frame1
#P1_0 = sp.Matrix([p_c(t)*sp.cos(phi(t)), p_c(t)*sp.sin(phi(t)), 0])  #CONFERIR
P1_0 = sp.Matrix([x_c(t), y_c(t), 0]) #CONFERIR

P1_0_dot = sp.simplify(sp.diff(P1_0, t))
display('P1_0: ', P1_0)
display('P1_0_dot: ', P1_0_dot)
print('-----------------------------------------')
# ........................
#Frame4
P4_1 = sp.Matrix([l*sp.sin(theta(t)), 0, l*sp.cos(theta(t))])
P4_0 = P1_0 + R1_0*P4_1
P4_0_dot = sp.simplify(sp.diff(P4_0, t))
display('P4_0: ', P4_0)
display('P4_0_dot: ', P4_0_dot)
print('-----------------------------------------')
# ........................
#Frame2
P2_1 = sp.Matrix([0, -0.5*d, 0])
P2_0 = P1_0 + R1_0*P2_1
P2_0_dot = sp.simplify(sp.diff(P2_0, t))
display('P2_0: ', P2_0)
display('P2_0_dot: ', P2_0_dot)
print('-----------------------------------------')
# ........................
#Frame3
P3_1 = sp.Matrix([0, 0.5*d, 0])
P3_0 = P1_0 + R1_0*P3_1
P3_0_dot = sp.simplify(sp.diff(P3_0, t))
display('P3_0: ', P3_0)
display('P3_0_dot: ', P3_0_dot)
print('-----------------------------------------')
# ........................

#------------------------------------------------

'P1_0: '

Matrix([
[x_c(t)],
[y_c(t)],
[     0]])

'P1_0_dot: '

Matrix([
[Derivative(x_c(t), t)],
[Derivative(y_c(t), t)],
[                    0]])

-----------------------------------------


...

'P3_0: '

Matrix([
[-0.5*d*sin(phi(t)) + x_c(t)],
[ 0.5*d*cos(phi(t)) + y_c(t)],
[                          0]])

'P3_0_dot: '

Matrix([
[-0.5*d*cos(phi(t))*Derivative(phi(t), t) + Derivative(x_c(t), t)],
[-0.5*d*sin(phi(t))*Derivative(phi(t), t) + Derivative(y_c(t), t)],
[                                                               0]])

-----------------------------------------


In [17]:
#--------------- Total Kinetic Energy (K) ---------------
K_trans = 0.5*m_w*sp.transpose(P2_0_dot)*P2_0_dot + 0.5*m_w*sp.transpose(P3_0_dot)*P3_0_dot + 0.5*m_b*sp.transpose(P4_0_dot)*P4_0_dot
K_trans =  sp.simplify(K_trans[0])

K_rot = 0.5*sp.transpose(w2_0)*I_w*w2_0 + 0.5*sp.transpose(w3_0)*I_w*w3_0 + 0.5*sp.transpose(w4_0)*I_b*w4_0
K_rot = sp.simplify(K_rot[0])

K =  K_trans+K_rot
#------------------------------------------------

#--------------- Total Potential Energy (K) ---------------
V = m_b*g*l*sp.cos(theta(t))
#------------------------------------------------

#--------------- Lagrangian (L) ---------------
L = K - V
display(sp.simplify(L))
#------------------------------------------------

#--------------- Exernal Torques (L) ---------------
tau_R = sp.Symbol('tau_R', real=True)
tau_L = sp.Symbol('tau_L', real=True)
Q = sp.Matrix([[0],\
                 [0],\
                 [0],\
                 [-tau_L-tau_R],\
                 [tau_R],\
                 [tau_L]
                 ])
#------------------------------------------------


#--------------- Euler_Lagrange Equation of Motion (E_L) ---------------
dL_dq_dot = sp.simplify(sp.diff(L, q_dot))
E_L = sp.simplify(sp.diff(dL_dq_dot,t) - sp.diff(L, q))
display(E_L)


0.5*Ib1*sin(theta(t))**2*Derivative(phi(t), t)**2 + 0.5*Ib2*Derivative(theta(t), t)**2 - 0.5*Ib3*sin(theta(t))**2*Derivative(phi(t), t)**2 + 0.5*Ib3*Derivative(phi(t), t)**2 + 0.5*J*Derivative(alpha_L(t), t)**2 + 0.5*J*Derivative(alpha_R(t), t)**2 + 1.0*K*Derivative(phi(t), t)**2 + 0.25*d**2*m_w*Derivative(phi(t), t)**2 - g*l*m_b*cos(theta(t)) + 0.5*l**2*m_b*sin(theta(t))**2*Derivative(phi(t), t)**2 + 0.5*l**2*m_b*Derivative(theta(t), t)**2 - 1.0*l*m_b*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t)*Derivative(x_c(t), t) + 1.0*l*m_b*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t)*Derivative(y_c(t), t) + 1.0*l*m_b*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t)*Derivative(y_c(t), t) + 1.0*l*m_b*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t)*Derivative(x_c(t), t) + 0.5*m_b*Derivative(x_c(t), t)**2 + 0.5*m_b*Derivative(y_c(t), t)**2 + 1.0*m_w*Derivative(x_c(t), t)**2 + 1.0*m_w*Derivative(y_c(t), t)**2

Matrix([
[                                                                                                                                                                                                                     -1.0*l*m_b*sin(phi(t))*sin(theta(t))*Derivative(phi(t), (t, 2)) - 2.0*l*m_b*sin(phi(t))*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) - 1.0*l*m_b*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t)**2 - 1.0*l*m_b*sin(theta(t))*cos(phi(t))*Derivative(theta(t), t)**2 + 1.0*l*m_b*cos(phi(t))*cos(theta(t))*Derivative(theta(t), (t, 2)) + 1.0*m_b*Derivative(x_c(t), (t, 2)) + 2.0*m_w*Derivative(x_c(t), (t, 2))],
[                                                                                                                                                                                                                     -1.0*l*m_b*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t)**2 - 1.0*l*m_b*sin(phi(t))*sin(theta(t))*Derivative(theta(t), t)**2 + 1.0*l*m_b*sin(p

In [18]:
print('-------------------------------------')
print("Equation 1:")
display(sp.collect(sp.collect(E_L[0], l*sp.sin(theta(t))*sp.cos(phi(t))), m_b))
print('-------------------------------------')
print("Equation 2:")
display(sp.collect(sp.collect(E_L[1], l*sp.sin(theta(t))*sp.sin(phi(t))), m_b))
print('-------------------------------------')
print("Equation 3:")
display(sp.collect(E_L[2], [ sp.diff(sp.diff(theta(t),t),t), sp.diff(phi(t), t)*sp.diff(phi(t), t)*sp.sin(2*theta(t))]))
print('-------------------------------------')
print("Equation 4:")
display(sp.collect(E_L[3], [sp.diff(sp.diff(phi(t),t),t), m_b*l*sp.sin(theta(t))]))
print('-------------------------------------')
#------------------------------------------------

-------------------------------------
Equation 1:


m_b*(l*(-1.0*Derivative(phi(t), t)**2 - 1.0*Derivative(theta(t), t)**2)*sin(theta(t))*cos(phi(t)) - 1.0*l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), (t, 2)) - 2.0*l*sin(phi(t))*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) + 1.0*l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), (t, 2)) + 1.0*Derivative(x_c(t), (t, 2))) + 2.0*m_w*Derivative(x_c(t), (t, 2))

-------------------------------------
Equation 2:


m_b*(l*(-1.0*Derivative(phi(t), t)**2 - 1.0*Derivative(theta(t), t)**2)*sin(phi(t))*sin(theta(t)) + 1.0*l*sin(phi(t))*cos(theta(t))*Derivative(theta(t), (t, 2)) + 1.0*l*sin(theta(t))*cos(phi(t))*Derivative(phi(t), (t, 2)) + 2.0*l*cos(phi(t))*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) + 1.0*Derivative(y_c(t), (t, 2))) + 2.0*m_w*Derivative(y_c(t), (t, 2))

-------------------------------------
Equation 3:


1.0*Ib1*sin(theta(t))**2*Derivative(phi(t), (t, 2)) + 1.0*Ib1*sin(2*theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) - 1.0*Ib3*sin(theta(t))**2*Derivative(phi(t), (t, 2)) - 1.0*Ib3*sin(2*theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) + 1.0*Ib3*Derivative(phi(t), (t, 2)) + 2.0*K*Derivative(phi(t), (t, 2)) + 0.5*d**2*m_w*Derivative(phi(t), (t, 2)) + 1.0*l**2*m_b*sin(theta(t))**2*Derivative(phi(t), (t, 2)) + 1.0*l**2*m_b*sin(2*theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) - 1.0*l*m_b*sin(phi(t))*sin(theta(t))*Derivative(x_c(t), (t, 2)) + 1.0*l*m_b*sin(theta(t))*cos(phi(t))*Derivative(y_c(t), (t, 2))

-------------------------------------
Equation 4:


-0.5*Ib1*sin(2*theta(t))*Derivative(phi(t), t)**2 + 1.0*Ib2*Derivative(theta(t), (t, 2)) + 0.5*Ib3*sin(2*theta(t))*Derivative(phi(t), t)**2 - 1.0*g*l*m_b*sin(theta(t)) - 0.5*l**2*m_b*sin(2*theta(t))*Derivative(phi(t), t)**2 + 1.0*l**2*m_b*Derivative(theta(t), (t, 2)) + 1.0*l*m_b*sin(phi(t))*cos(theta(t))*Derivative(y_c(t), (t, 2)) + 1.0*l*m_b*cos(phi(t))*cos(theta(t))*Derivative(x_c(t), (t, 2))

-------------------------------------


In [19]:
#--------------- Canonical Equations ---------------
def get_Mass_Matrix(q_, L_):
  M = sp.zeros(len(q_),len(q_))

  for i in range(0,len(q_)):
    for j in range(0,len(q_)):
      M[i, j] =  sp.diff(sp.diff(L_,sp.diff(q[i],t,1)),sp.diff(q[j],t,1))
  display(M)
  return M

M = get_Mass_Matrix(q, L)
G_C = sp.Matrix([V]).jacobian(q).transpose()


C2 = sp.zeros(len(q), len(q))

for k in range(0,len(q)):
  for j in range(0, len(q)):
    #C2[k,j] = 0
    for i in range(0, len(q)):
      C2[k,j] += ( sp.diff(M[k,j], q[i]) - 0.5*sp.diff(M[j,i], q[k]) )*q_dot[i]

display(C2)

C2 = sp.transpose(sp.Matrix([[K]]).jacobian(q_dot)).jacobian(q) * q_dot - sp.transpose(sp.Matrix([[K]]).jacobian(q))

display(G_C)
display(sp.simplify(C2))
#------------------------------------------------

Matrix([
[                   1.0*m_b + 2.0*m_w,                                   0,                                                                                 -1.0*l*m_b*sin(phi(t))*sin(theta(t)), 1.0*l*m_b*cos(phi(t))*cos(theta(t)),     0,     0],
[                                   0,                   1.0*m_b + 2.0*m_w,                                                                                  1.0*l*m_b*sin(theta(t))*cos(phi(t)), 1.0*l*m_b*sin(phi(t))*cos(theta(t)),     0,     0],
[-1.0*l*m_b*sin(phi(t))*sin(theta(t)), 1.0*l*m_b*sin(theta(t))*cos(phi(t)), 1.0*Ib1*sin(theta(t))**2 - 1.0*Ib3*sin(theta(t))**2 + 1.0*Ib3 + 2.0*K + 0.5*d**2*m_w + 1.0*l**2*m_b*sin(theta(t))**2,                                   0,     0,     0],
[ 1.0*l*m_b*cos(phi(t))*cos(theta(t)), 1.0*l*m_b*sin(phi(t))*cos(theta(t)),                                                                                                                    0,              1.0*Ib2 + 1.0*l**2*m_b,     0,     0],
[      

Matrix([
[                                                                                                                       0,                                                                                                                        0,                                                                                                                                               -1.0*l*m_b*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t) - 1.0*l*m_b*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t), -1.0*l*m_b*sin(phi(t))*cos(theta(t))*Derivative(phi(t), t) - 1.0*l*m_b*sin(theta(t))*cos(phi(t))*Derivative(theta(t), t), 0, 0],
[                                                                                                                       0,                                                                                                                        0,                                                                                                             

Matrix([
[                     0],
[                     0],
[                     0],
[-g*l*m_b*sin(theta(t))],
[                     0],
[                     0]])

Matrix([
[-l*m_b*(2.0*sin(phi(t))*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) + 1.0*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t)**2 + 1.0*sin(theta(t))*cos(phi(t))*Derivative(theta(t), t)**2)],
[l*m_b*(-1.0*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t)**2 - 1.0*sin(phi(t))*sin(theta(t))*Derivative(theta(t), t)**2 + 2.0*cos(phi(t))*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t))],
[                                                                                                                1.0*(Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t)],
[                                                                                                                                    0.5*(-Ib1 + Ib3 - l**2*m_b)*sin(2*theta(t))*Derivative(phi(t), t)**2],
[                                                                                                                                                                              

In [20]:
#--------------- Non Holonomic Constraints ---------------
#No forward slip and no lateral slip conditions
#Set of constraints:

constraints = sp.Matrix([
     [sp.diff(x_c(t),t)*sp.sin(phi(t)) - sp.diff(y_c(t),t)*sp.cos(phi(t))],\
     [sp.diff(x_c(t),t)*sp.cos(phi(t)) + sp.diff(y_c(t),t)*sp.sin(phi(t)) + (d/2)*sp.diff(phi(t),t) - r*sp.diff(alpha_R(t),t)],\
     [sp.diff(x_c(t),t)*sp.cos(phi(t)) + sp.diff(y_c(t),t)*sp.sin(phi(t)) - (d/2)*sp.diff(phi(t),t) - r*sp.diff(alpha_L(t),t)]
                        ])


display(constraints)

A = sp.zeros(len(constraints),len(q))
for i in range(0, len(constraints)):
  for j in range(0, len(q_dot)):
    A[i, j] = sp.diff(constraints[i], q_dot[j])


G = sp.Matrix([[sp.cos(phi(t)), 0, 0],\
               [sp.sin(phi(t)), 0, 0],\
               [0, 1, 0],\
               [0, 0, 1],\
               [1/r, d/(2*r), 0],\
               [1/r, -d/(2*r), 0]
               ])
G_dot = sp.diff(G, t)


display('A', A)
display('G', G)
display(G_dot)
display(sp.simplify(A*G)) #this must be zero!

Matrix([
[                                                           sin(phi(t))*Derivative(x_c(t), t) - cos(phi(t))*Derivative(y_c(t), t)],
[ d*Derivative(phi(t), t)/2 - r*Derivative(alpha_R(t), t) + sin(phi(t))*Derivative(y_c(t), t) + cos(phi(t))*Derivative(x_c(t), t)],
[-d*Derivative(phi(t), t)/2 - r*Derivative(alpha_L(t), t) + sin(phi(t))*Derivative(y_c(t), t) + cos(phi(t))*Derivative(x_c(t), t)]])

'A'

Matrix([
[sin(phi(t)), -cos(phi(t)),    0, 0,  0,  0],
[cos(phi(t)),  sin(phi(t)),  d/2, 0, -r,  0],
[cos(phi(t)),  sin(phi(t)), -d/2, 0,  0, -r]])

'G'

Matrix([
[cos(phi(t)),        0, 0],
[sin(phi(t)),        0, 0],
[          0,        1, 0],
[          0,        0, 1],
[        1/r,  d/(2*r), 0],
[        1/r, -d/(2*r), 0]])

Matrix([
[-sin(phi(t))*Derivative(phi(t), t), 0, 0],
[ cos(phi(t))*Derivative(phi(t), t), 0, 0],
[                                 0, 0, 0],
[                                 0, 0, 0],
[                                 0, 0, 0],
[                                 0, 0, 0]])

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [21]:
#New canonical equations
#................Pseudo-velocity vector
x = sp.Function('x', real=True) #Forward velocity v_c
x_dot = sp.diff(x(t), t)

v = sp.Matrix([x_dot, sp.diff(phi(t),t), sp.diff(theta(t),t)]) #pseudo-velocity


v_dot = sp.diff(v, t)
display('v', v)

Gv = G*v
display('G', G)
display('Gv', sp.simplify(Gv))
#......................................

M_G = sp.simplify(sp.transpose(G)*M*G)
C_G = sp.simplify(sp.transpose(G)*M*G_dot)
#H_G = sp.simplify(sp.transpose(G)*C2*q_dot)#CONFERIR
H_G = sp.simplify(sp.transpose(G)*C2)#SMS
G_G = sp.simplify(sp.transpose(G)*G_C)
Q_G = sp.simplify(sp.transpose(G)*Q)
E_L2 = sp.simplify(M_G*v_dot + (C_G)*v +H_G + G_G)

display('M_G', M_G)
display('C_G', C_G)
display('H_G', H_G)
display('G_G', G_G)
display('Q_G', Q_G)
display('E_L2', E_L2)
display(sp.simplify((C_G)*v +H_G))


'v'

Matrix([
[    Derivative(x(t), t)],
[  Derivative(phi(t), t)],
[Derivative(theta(t), t)]])

'G'

Matrix([
[cos(phi(t)),        0, 0],
[sin(phi(t)),        0, 0],
[          0,        1, 0],
[          0,        0, 1],
[        1/r,  d/(2*r), 0],
[        1/r, -d/(2*r), 0]])

'Gv'

...

'Q_G'

Matrix([
[       (tau_L + tau_R)/r],
[d*(-tau_L + tau_R)/(2*r)],
[          -tau_L - tau_R]])

'E_L2'

Matrix([
[                                                                                                               (1.0*l*m_b*r**2*(-(Derivative(phi(t), t)**2 + Derivative(theta(t), t)**2)*sin(theta(t)) + cos(theta(t))*Derivative(theta(t), (t, 2))) + (2.0*J + r**2*(1.0*m_b + 2.0*m_w))*Derivative(x(t), (t, 2)))/r**2],
[(1.0*r**2*(l*m_b*sin(theta(t))*Derivative(x(t), t) + (Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(theta(t), t))*Derivative(phi(t), t) + (0.5*J*d**2 + r**2*(1.0*Ib1*sin(theta(t))**2 - 1.0*Ib3*sin(theta(t))**2 + 1.0*Ib3 + 2.0*K + 0.5*d**2*m_w + 1.0*l**2*m_b*sin(theta(t))**2))*Derivative(phi(t), (t, 2)))/r**2],
[                                                                                                                     -g*l*m_b*sin(theta(t)) + 1.0*l*m_b*cos(theta(t))*Derivative(x(t), (t, 2)) + 1.0*(Ib2 + l**2*m_b)*Derivative(theta(t), (t, 2)) - 0.5*(Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(phi(t), t)**2]])

Matrix([
[                                                    -1.0*l*m_b*(Derivative(phi(t), t)**2 + Derivative(theta(t), t)**2)*sin(theta(t))],
[1.0*(l*m_b*sin(theta(t))*Derivative(x(t), t) + (Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(theta(t), t))*Derivative(phi(t), t)],
[                                                                0.5*(-Ib1 + Ib3 - l**2*m_b)*sin(2*theta(t))*Derivative(phi(t), t)**2]])

In [22]:
print('-------------------------------------')
print("Equation 1:")
display(sp.collect(E_L2[0], [sp.diff(x_dot,t), -m_b*l*sp.sin(theta(t)), sp.diff(theta(t),t,2)]))
print('-------------------------------------')
print("Equation 2:")
display(sp.collect(E_L2[1], [sp.diff(theta(t),t,2), sp.diff(x_dot,t), sp.diff(phi(t),t)**2*sp.sin(theta(t))*sp.cos(theta(t))]))
print('-------------------------------------')
print("Equation 3:")
display(sp.collect(E_L2[2], [sp.diff(phi(t),t,2), sp.diff(phi(t),t)*sp.sin(theta(t))]))
print('-------------------------------------')


-------------------------------------
Equation 1:


(1.0*l*m_b*r**2*(-(Derivative(phi(t), t)**2 + Derivative(theta(t), t)**2)*sin(theta(t)) + cos(theta(t))*Derivative(theta(t), (t, 2))) + (2.0*J + r**2*(1.0*m_b + 2.0*m_w))*Derivative(x(t), (t, 2)))/r**2

-------------------------------------
Equation 2:


(1.0*r**2*(l*m_b*sin(theta(t))*Derivative(x(t), t) + (Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(theta(t), t))*Derivative(phi(t), t) + (0.5*J*d**2 + r**2*(1.0*Ib1*sin(theta(t))**2 - 1.0*Ib3*sin(theta(t))**2 + 1.0*Ib3 + 2.0*K + 0.5*d**2*m_w + 1.0*l**2*m_b*sin(theta(t))**2))*Derivative(phi(t), (t, 2)))/r**2

-------------------------------------
Equation 3:


-g*l*m_b*sin(theta(t)) + 1.0*l*m_b*cos(theta(t))*Derivative(x(t), (t, 2)) + 1.0*(Ib2 + l**2*m_b)*Derivative(theta(t), (t, 2)) - 0.5*(Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(phi(t), t)**2

-------------------------------------


In [23]:
#Recalculating the Canonical equations, now considering q = [x_c, y_c theta, psi]^T
q2 = sp.Matrix([[x_c(t)],\
               [y_c(t)],\
                [phi(t)],\
                [theta(t)]
                ])
q2_dot = sp.diff(q2, t)
#------------------------------------------------------------------------------
M_G_inv = M_G.inv()

u = sp.Matrix([[tau_R],\
               [tau_L]
                ])


In [24]:
#Dynamic Model Linearization--------------------------------------------------
#1) Representing the equations of motion in the form x_dot = a(x) + b(x)u
x = sp.Matrix([[q2],\
               [v]])
display(x)
vd_dot = sp.simplify(M_G_inv*( -(C_G)*v - H_G - G_G +Q_G))
display(vd_dot)

#x_dot = f(x) + g1(x)u
E = Q.jacobian(u)
display(E)
g1 = sp.Matrix([sp.zeros(len(q2), len(u)),\
               sp.simplify(M_G_inv*sp.transpose(G)*E)
                ])
Gr = G[0:4,:]
f = sp.Matrix([Gr*v,\
               sp.simplify(-M_G_inv*((C_G)*v + H_G + G_G))
                ])

A = f.jacobian(x)
A_linearized =  sp.simplify(A.subs([(theta(t), 0), (phi(t),0), (x_dot,0)]))#sp.simplify(A.subs([(theta(t), 0), (sp.diff(phi(t),t),0), (x_dot,0),( sp.cos(phi(t)), phi(t)),( sp.sin(phi(t)), phi(t))]))
#A_linearized = sp.simplify(A.subs(theta(t), 0))
B_linearized = sp.simplify(g1.subs(theta(t), 0))
display(A_linearized)
display(B_linearized)

Matrix([
[                 x_c(t)],
[                 y_c(t)],
[                 phi(t)],
[               theta(t)],
[    Derivative(x(t), t)],
[  Derivative(phi(t), t)],
[Derivative(theta(t), t)]])

Matrix([
[0.25*r*(-l*m_b*r*(g*l*m_b*sin(theta(t)) - tau_L - tau_R + 0.5*(Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(phi(t), t)**2)*cos(theta(t)) + (Ib2 + l**2*m_b)*(1.0*l*m_b*r*(Derivative(phi(t), t)**2 + Derivative(theta(t), t)**2)*sin(theta(t)) + tau_L + tau_R))*(2.0*Ib2*J + 1.0*Ib2*m_b*r**2 + 2.0*Ib2*m_w*r**2 + 2.0*J*l**2*m_b + 1.0*l**2*m_b**2*r**2*sin(theta(t))**2 + 2.0*l**2*m_b*m_w*r**2)/(1.0*Ib2*J + 0.5*Ib2*m_b*r**2 + 1.0*Ib2*m_w*r**2 + 1.0*J*l**2*m_b + 0.5*l**2*m_b**2*r**2*sin(theta(t))**2 + 1.0*l**2*m_b*m_w*r**2)**2],
[                                                                                                                                                                                                        -0.5*r*(d*(tau_L - tau_R) + 2.0*r*(l*m_b*sin(theta(t))*Derivative(x(t), t) + (Ib1 - Ib3 + l**2*m_b)*sin(2*theta(t))*Derivative(theta(t), t))*Derivative(phi(t), t))/(1.0*Ib1*r**2*sin(theta(t))**2 - 1.0*Ib3*r**2*sin(theta(t))**2 + 1.0*Ib3*r**2 + 0.5*J*d**2 + 2.0*

Matrix([
[ 0,  0],
[ 0,  0],
[ 0,  0],
[-1, -1],
[ 1,  0],
[ 0,  1]])

Matrix([
[0, 0, 0,                                                                                                                                        0, 1, 0, 0],
[0, 0, 0,                                                                                                                                        0, 0, 0, 0],
[0, 0, 0,                                                                                                                                        0, 0, 1, 0],
[0, 0, 0,                                                                                                                                        0, 0, 0, 1],
[0, 0, 0,                       -1.0*g*l**2*m_b**2*r**2/(2.0*Ib2*J + 1.0*Ib2*m_b*r**2 + 2.0*Ib2*m_w*r**2 + 2.0*J*l**2*m_b + 2.0*l**2*m_b*m_w*r**2), 0, 0, 0],
[0, 0, 0,                                                                                                                                        0, 0, 0, 0],
[0, 0, 0, g*l*m_b*(2.0*J + 1.0*m_b*r**2 + 2

Matrix([
[                                                                                                                                              0,                                                                                                                                               0],
[                                                                                                                                              0,                                                                                                                                               0],
[                                                                                                                                              0,                                                                                                                                               0],
[                                                                                                                  

In [25]:
#Solving Lagranges Equations by assuming they are equal to zero
sols = sp.solve(E_L2-Q_G, v_dot, simplify=False, rational=False)
display(sols)

{Derivative(phi(t), (t, 2)): -2.0*Ib1*r**2*sin(2*theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t)/(2.0*Ib1*r**2*sin(theta(t))**2 - 2.0*Ib3*r**2*sin(theta(t))**2 + 2.0*Ib3*r**2 + 1.0*J*d**2 + 4.0*K*r**2 + 1.0*d**2*m_w*r**2 + 2.0*l**2*m_b*r**2*sin(theta(t))**2) + 2.0*Ib3*r**2*sin(2*theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t)/(2.0*Ib1*r**2*sin(theta(t))**2 - 2.0*Ib3*r**2*sin(theta(t))**2 + 2.0*Ib3*r**2 + 1.0*J*d**2 + 4.0*K*r**2 + 1.0*d**2*m_w*r**2 + 2.0*l**2*m_b*r**2*sin(theta(t))**2) - 1.0*d*r*tau_L/(2.0*Ib1*r**2*sin(theta(t))**2 - 2.0*Ib3*r**2*sin(theta(t))**2 + 2.0*Ib3*r**2 + 1.0*J*d**2 + 4.0*K*r**2 + 1.0*d**2*m_w*r**2 + 2.0*l**2*m_b*r**2*sin(theta(t))**2) + 1.0*d*r*tau_R/(2.0*Ib1*r**2*sin(theta(t))**2 - 2.0*Ib3*r**2*sin(theta(t))**2 + 2.0*Ib3*r**2 + 1.0*J*d**2 + 4.0*K*r**2 + 1.0*d**2*m_w*r**2 + 2.0*l**2*m_b*r**2*sin(theta(t))**2) - 2.0*l**2*m_b*r**2*sin(2*theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t)/(2.0*Ib1*r**2*sin(theta(t))**2 - 2.0*Ib3*r**2*sin(theta(